In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests


/opt/anaconda3/envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('wjbmattingly/trocr-yiddish')

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

In [13]:
# load image from the IAM database
url = 'https://datasets-server.huggingface.co/cached-assets/johnlockejrr/yiddish_synth/--/cf3f57fb05623c211a5beed297e591810bc6891f/--/default/train/48/image/image.jpg?Expires=1745259392&Signature=yPYjjePauuEvs1EuklCo~PQ5bGh4pw0OEB4ywYb49rJC3BaUG-7wvy~EBSo2N784BBDd5R5O4U6OIDkAEI2LBDyEgI0tAHihUJ5wjx~~0ya~sonqWJ7uMoPl36i5f8mjMxoSdp8NGrWW-18TdaoETCzFlTBl40TP-pBb3rOvOv2tAuEELs7qBhL5IEndDB0Ye6gwWriWSEV8mkwaeBPb2uoxKk-L4nLAgAy7o6KiZI8EFcrpIk29-K~cwClTP65F-4WCKyiEghA2EoQIX0Q-dm-UMHh-83B6BB3yWzijYx6W7N8S8X4Kl8rgCBJAtcvZceYnK1grqHdAnQjQjulS0A__&Key-Pair-Id=K3EI6M078Z3AC3'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
print(generated_text)
print("הערשלס טאַטע און זלמן דער שענקער פֿון")

</s>�ה�����ו�</s>
הערשלס טאַטע און זלמן דער שענקער פֿון


In [12]:
with open("test.txt", "w", encoding="utf-8") as f:
    f.write(generated_text)

In [15]:
# Add this to your notebook
# Checking actual characters in the generated text
print("Character codes in generated text:")
for char in generated_text:
    print(f"{char}: {ord(char)} (hex: {hex(ord(char))}")

# Try with skip_special_tokens=True
print("\nAttempting with skip_special_tokens=True:")
clean_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(clean_text)

# Create a custom processor that loads the same tokenizer from the finetuned model
from transformers import TrOCRProcessor

# Try creating a processor from the same finetuned model path
custom_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
custom_text = custom_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("\nUsing tokenizer from the same model:")
print(custom_text)

Character codes in generated text:
<: 60 (hex: 0x3c
/: 47 (hex: 0x2f
s: 115 (hex: 0x73
>: 62 (hex: 0x3e
�: 65533 (hex: 0xfffd
ה: 1492 (hex: 0x5d4
�: 65533 (hex: 0xfffd
�: 65533 (hex: 0xfffd
�: 65533 (hex: 0xfffd
�: 65533 (hex: 0xfffd
�: 65533 (hex: 0xfffd
ו: 1493 (hex: 0x5d5
�: 65533 (hex: 0xfffd
<: 60 (hex: 0x3c
/: 47 (hex: 0x2f
s: 115 (hex: 0x73
>: 62 (hex: 0x3e

Attempting with skip_special_tokens=True:
�ה�����ו�

Using tokenizer from the same model:
�ה�����ו�


In [17]:
# Load your custom processor and model
processor = TrOCRProcessor.from_pretrained('yiddish_processor')
model = VisionEncoderDecoderModel.from_pretrained('wjbmattingly/trocr-yiddish')

# Update model to use your tokenizer
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.eos_token_id = processor.tokenizer.sep_token_id

# Resize model's token embeddings to match your tokenizer
model.decoder.resize_token_embeddings(len(processor.tokenizer))

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

TrOCRScaledWordEmbedding(23437, 1024, padding_idx=1)

In [18]:
# Now process the image
url = 'https://datasets-server.huggingface.co/cached-assets/johnlockejrr/yiddish_synth/--/cf3f57fb05623c211a5beed297e591810bc6891f/--/default/train/48/image/image.jpg?Expires=1745259392&Signature=yPYjjePauuEvs1EuklCo~PQ5bGh4pw0OEB4ywYb49rJC3BaUG-7wvy~EBSo2N784BBDd5R5O4U6OIDkAEI2LBDyEgI0tAHihUJ5wjx~~0ya~sonqWJ7uMoPl36i5f8mjMxoSdp8NGrWW-18TdaoETCzFlTBl40TP-pBb3rOvOv2tAuEELs7qBhL5IEndDB0Ye6gwWriWSEV8mkwaeBPb2uoxKk-L4nLAgAy7o6KiZI8EFcrpIk29-K~cwClTP65F-4WCKyiEghA2EoQIX0Q-dm-UMHh-83B6BB3yWzijYx6W7N8S8X4Kl8rgCBJAtcvZceYnK1grqHdAnQjQjulS0A__&Key-Pair-Id=K3EI6M078Z3AC3'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# Process with your custom processor
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Generate text
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

קאָלעהודמעמע זעגטשפּרונגען קאָלעהודשפּרונגען
